In [64]:
from skyfield.api import load, wgs84, Angle
import pickle
from skyfield.api import  Topos
from skyfield.positionlib import ICRF
from skyfield.data import mpc
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN
import pandas as pd

# Planets

In [12]:
planets=[
    'mercury barycenter',
    'venus barycenter',
    'mars barycenter',
    'jupiter barycenter',
    'saturn barycenter',
    'uranus barycenter',
    'neptune barycenter',
    'pluto barycenter'
]
ts = load.timescale()
time_p = ts.utc(2018, 1, range(1,365*3+2),12,20)

In [3]:
time_p[365*3].utc_jpl()

'A.D. 2020-Dec-31 12:20:00.0000 UTC'

In [4]:
eph = load('de421.bsp')

[#################################] 100% de421.bsp


In [13]:
earth = eph['earth']
#moon = eph['moon']
gangtok = earth + wgs84.latlon( 27.3314 , 88.6138)
gg = gangtok.at(time_p)
P=[]
for i in planets:
    P.append(eph[i])

In [6]:
astro = gg.observe(P[0])
lat, lon, dist = astro.ecliptic_latlon()

In [43]:
sun = eph['sun']

In [9]:
lat.degrees[0]

1.934034596822139

In [14]:
planet_c_dist = {}

for planet, p_name in zip(P, planets):
    print(p_name," done ")
    astronometric = gg.observe(planet)
    lat, lon, dist = astronometric.ecliptic_latlon()
    eph_planet = {'lat' : lat,
                  'lon' : lon,
                 'distance' : dist}
    planet_c_dist[p_name[:-11]]= eph_planet

mercury barycenter  done 
venus barycenter  done 
mars barycenter  done 
jupiter barycenter  done 
saturn barycenter  done 
uranus barycenter  done 
neptune barycenter  done 
pluto barycenter  done 


In [23]:
planet_c_dist['pluto']

{'lat': <Angle 1096 values from +00deg 27' 13.2" to -01deg 11' 31.8">,
 'lon': <Angle 1096 values from 288deg 33' 13.7" to 293deg 53' 20.0">,
 'distance': <Distance [34.44992545 34.45276288 34.45531009 ... 35.13462316 35.13987256
  35.14483876] au>}

In [24]:
planet_names=[
    'mercury',
    'venus',
    'mars',
    'jupiter',
    'saturn',
    'uranus',
    'neptune',
    'pluto'
]

In [51]:
latitude={}
longitude={}
distance={}
for name in planet_names:
    latitude[name]=planet_c_dist[name]['lat'].degrees
    longitude[name]=planet_c_dist[name]['lon'].degrees
    distance[name]=planet_c_dist[name]['distance'].au

In [52]:
latitude['time']=time
longitude['time']=time
distance['time']=time

In [31]:
time=time_p.utc_jpl()

In [33]:
time[365*3]

'A.D. 2020-Dec-31 12:20:00.0000 UTC'

# ceres

In [40]:
with load.open('MPCORB.excerpt.DAT') as f:
    minor_planets = mpc.load_mpcorb_dataframe(f)

print(minor_planets.shape[0], 'minor planets loaded')

4 minor planets loaded


In [41]:
row_ceres=minor_planets.iloc[0]

In [58]:
ceres = sun + mpc.mpcorb_orbit(row_ceres, ts, GM_SUN)
lat, lon, dis = earth.at(time_p).observe(ceres).ecliptic_latlon()
print(lat)
print(lon)
print(dis.au)

1096 values from +10deg 55' 07.1" to -09deg 24' 19.3"
1096 values from 136deg 08' 32.7" to 341deg 33' 25.3"
[1.71957871 1.71186367 1.70436454 ... 3.26704838 3.27918451 3.29124125]


In [75]:
latitude['ceres']=lat.degrees
longitude['ceres']=lon.degrees
distance['ceres']=dis.au

In [71]:
latitude[çeres]

array([10.91864421, 11.01695558, 11.11457222, ..., -9.48685682,
       -9.44589305, -9.40535807])

In [61]:
col_names=[
    'time',
    'mercury',
    'venus',
    'mars',
    'jupiter',
    'saturn',
    'uranus',
    'neptune',
    'pluto',
    'ceres'
]

In [76]:
lats = pd.DataFrame(latitude, columns=col_names)
longs = pd.DataFrame(longitude, columns=col_names)
dists = pd.DataFrame(distance, columns=col_names)

In [82]:
lats

,time,mercury,venus,mars,jupiter,saturn,uranus,neptune,pluto,ceres
0,A.D. 2018-Jan-01 12:20:00.0000 UTC,1.934035,-0.488559,0.897383,1.052536,0.898481,-0.570919,-0.911828,0.453678,10.918644
1,A.D. 2018-Jan-02 12:20:00.0000 UTC,1.791378,-0.525848,0.890939,1.054131,0.897632,-0.570327,-0.911530,0.452116,11.016956
2,A.D. 2018-Jan-03 12:20:00.0000 UTC,1.646749,-0.562728,0.884391,1.055756,0.896804,-0.569734,-0.911237,0.450559,11.114572
3,A.D. 2018-Jan-04 12:20:00.0000 UTC,1.500898,-0.599174,0.877739,1.057411,0.895995,-0.569140,-0.910949,0.449006,11.211420
4,A.D. 2018-Jan-05 12:20:00.0000 UTC,1.354466,-0.635162,0.870978,1.059095,0.895207,-0.568545,-0.910666,0.447456,11.307421
...,...,...,...,...,...,...,...,...,...,...
1091,A.D. 2020-Dec-27 12:20:00.0000 UTC,-1.936036,0.842084,0.775421,-0.483425,-0.381678,-0.454564,-1.082702,-1.186921,-9.570083
1092,A.D. 2020-Dec-28 12:20:00.0000 UTC,-1.982655,0.802290,0.797242,-0.484377,-0.382628,-0.454084,-1.082255,-1.188216,-9.528253
1093,A.D. 2020-Dec-29 12:20:00.0000 UTC,-2.023927,0.762081,0.818514,-0.485342,-0.383585,-0.453601,-1.081813,-1.189521,-9.486857
1094,A.D. 2020-Dec-30 12:20:00.0000 UTC,-2.059584,0.721485,0.839247,-0.486319,-0.384549,-0.453116,-1.081374,-1.190834,-9.445893


In [84]:
lats.to_csv('Latitudes.csv', index= False)
longs.to_csv('Longitudes.csv', index= False)
dists.to_csv('Distances.csv', index= False)

In [86]:
from skyfield.jpllib import SpiceKernel
#kernel = SpiceKernel('wld23593.15')

In [87]:
SpiceKernel

skyfield.jpllib.SpiceKernel